In [2]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path

## Import data 

In [3]:
# Import large kickstarter recent dataset
kickstarter_large_recent_df = pd.read_csv(
    Path('./Resources/kickstarter_data/ks-projects-201801.csv'),
)

# Imported data has a space at the end of the column name
# Remove spaces in columns name
kickstarter_large_recent_df.columns = kickstarter_large_recent_df.columns.str.replace(' ','')

# Set the index as the ID
kickstarter_large_recent_df.set_index('ID', inplace=True)

## Clean df

In [11]:
# Kickstarter code to fix dates
kickstarter_large_recent_df['launched'] = pd.to_datetime(kickstarter_large_recent_df['launched'])
kickstarter_large_recent_df['deadline'] = pd.to_datetime(kickstarter_large_recent_df['deadline'])

# Only pull out failed and successful cases to match indiegogo df
failure_success = ['failed','successful']

# Quick check to see if we have already renamed state col
if 'failed' in kickstarter_large_recent_df['state'].values:
    kickstarter_large_recent_df = kickstarter_large_recent_df[kickstarter_large_recent_df['state'].isin(failure_success)]
    # Rename successful -> 1 and failed -> 0 to match indiegogo
    kickstarter_large_recent_df['state'].replace({'failed': '0', 'successful': '1'}, inplace=True)
    # Fix types
    kickstarter_large_recent_df = kickstarter_large_recent_df.astype({'state': 'int64'})

# Sometimes projects are successful with no backers. Remove this data
kickstarter_large_recent_df.drop(kickstarter_large_recent_df[(kickstarter_large_recent_df['state'] == 1) & (kickstarter_large_recent_df['backers'] == 0)].index, inplace = True)

# Remove all countries with that have euro currency and country equal to N,0"
kickstarter_large_recent_df.drop(kickstarter_large_recent_df[(kickstarter_large_recent_df['currency'] == 'EUR') & (kickstarter_large_recent_df['country'] == 'N,0"')].index, inplace = True)

# Create a dictionary of country and currency pairs to fix other N,0" countries to right country
country_currency_df = kickstarter_large_recent_df.loc[:,['country', 'currency']]
country_currency_df.drop(kickstarter_large_recent_df[kickstarter_large_recent_df['country'] == 'N,0"'].index, inplace=True)
country_currency_df.drop_duplicates(inplace=True)
country_currency_df.set_index('currency', inplace = True)
currency_country_dict = country_currency_df.to_dict()['country']

# Function to clean up country col
def replace_N0(country, currency):
    if country == 'N,0"':
        return currency_country_dict[currency]
    else:
        return country

# Clean up all the N,0" values for countries using the currency_country_dict
kickstarter_large_recent_df['country'] = kickstarter_large_recent_df.apply(lambda row: replace_N0(row['country'], row['currency']), axis=1)

/Users/n8patteson/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':


## Create dfs from selected columns

In [12]:
# List of columns to drop
kickstarter_cols_drop = ['category', 'goal', 'pledged', 'usdpledged']

# Selected columns kickstarter df
kickstarter_selected_cols_large_recent_df = kickstarter_large_recent_df.drop(kickstarter_cols_drop, axis=1)

## Add new cols

In [48]:
# Duration
kickstarter_selected_cols_large_recent_df['duration'] = kickstarter_selected_cols_large_recent_df['deadline'] - kickstarter_selected_cols_large_recent_df['launched'] 
kickstarter_selected_cols_large_recent_df['duration'] = kickstarter_selected_cols_large_recent_df['duration'].dt.days

# Daily Goal 
kickstarter_selected_cols_large_recent_df['daily_goal'] = round(kickstarter_selected_cols_large_recent_df['usd_goal_real'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Daily Pledged
kickstarter_selected_cols_large_recent_df['daily_pledged'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Funded Percentage
kickstarter_selected_cols_large_recent_df['funded_percent'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['usd_goal_real'],4)

# Average Backer Per Day
kickstarter_selected_cols_large_recent_df['avg_backer_per_day'] = round(kickstarter_selected_cols_large_recent_df['backers'] / kickstarter_selected_cols_large_recent_df['duration'],2)

# Pledged Per Person
kickstarter_selected_cols_large_recent_df['pledged_per_person'] = round(kickstarter_selected_cols_large_recent_df['usd_pledged_real'] / kickstarter_selected_cols_large_recent_df['backers'],2)
kickstarter_selected_cols_large_recent_df['pledged_per_person'] = kickstarter_selected_cols_large_recent_df['pledged_per_person'].fillna(0)

# Add full country name col
%run ./consts.ipynb
kickstarter_selected_cols_large_recent_df['full_country_names'] = kickstarter_selected_cols_large_recent_df['country'].apply(lambda x: full_country_names_dict[x])

# Add lat/long depends on running ./Util/conts.ipynb first
kickstarter_selected_cols_large_recent_df['lat'] = kickstarter_selected_cols_large_recent_df['country'].apply(lambda x: country_lat_long_dict[x][0])
kickstarter_selected_cols_large_recent_df['long'] = kickstarter_selected_cols_large_recent_df['country'].apply(lambda x: country_lat_long_dict[x][1])

# Fix types
kickstarter_selected_cols_large_recent_df = kickstarter_selected_cols_large_recent_df.astype({"funded_percent": 'float', "avg_backer_per_day":'float', "pledged_per_person": 'float'})

KeyError: 'backers'

## Reorder cols

In [47]:
# Base column names for reordering
base_order = ['name', 'main_category', 'currency', 'usd_goal_real', 'usd_pledged_real', 'deadline', 'launched', 'state', 'funded_percent','duration', 'daily_goal', 'daily_pledged', 'country', 'full_country_names', 'lat', 'long']
kickstarter_order = base_order + ['avg_backer_per_day', 'pledged_per_person']

# Reorder each df
kickstarter_selected_cols_large_recent_df = kickstarter_selected_cols_large_recent_df[kickstarter_order]

## Write clean data to files

In [ ]:
kickstarter_selected_cols_large_recent_df.to_csv('./Resources/kickstarter_data_clean/ks-projects-large.csv')